INSTALLING NECESSARY LIBRARIES !!

In [2]:
!pip install pandas
!pip install -q scikit-learn sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


DATA LOADING

In [3]:
import pandas as pd

# Re-load the dataset with better parsing
df = pd.read_csv('dataset/synthetic_logs.csv')


# Check the first rows to confirm proper parsing
print(df.head())


             timestamp           source  \
0  2025/06/27 07:20:25        ModernCRM   
1      1/14/2025 23:07        ModernCRM   
2       1/17/2025 1:29  AnalyticsEngine   
3  2025-07-12 00:24:16         ModernHR   
4  2025-06-02 18:25:23    BillingSystem   

                                         log_message    target_label  \
0  nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1     Email service experiencing issues with sending  Critical Error   
2          Unauthorized access to data was attempted  Security Alert   
3  nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   

  complexity  
0       bert  
1       bert  
2       bert  
3       bert  
4       bert  


In [4]:
df

,timestamp,source,log_message,target_label,complexity
0,2025/06/27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2421,1/10/2025 23:50,ModernHR,Detected unauthorized access to confidential H...,Security Alert,bert
2422,2025/12/05 15:45:12,AnalyticsEngine,"File processing completed. Total records: 10,4...",System Notification,bert
2423,2025/11/01 22:14:03,PaymentGateway,Transaction ID TX7845 failed due to insufficie...,Critical Error,bert
2424,1/23/2025 10:15:33,SecuritySystem,Firewall rule added: Restrict traffic to IP 19...,Security Alert,bert


In [5]:
df.head(10)

,timestamp,source,log_message,target_label,complexity
0,2025/06/27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert


In [6]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM', 'Linux', 'PaymentGateway',
       'SecuritySystem'], dtype=object)

In [7]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning', 'Warning'], dtype=object)

In [8]:
df[df.target_label=='System Notification'].sample(10)

,timestamp,source,log_message,target_label,complexity
880,6/20/2025 14:45,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex
1928,4/20/2025 13:46,ModernCRM,Backup completed successfully.,System Notification,regex
260,12/22/2025 1:34,ThirdPartyAPI,System updated to version 2.6.2.,System Notification,regex
1688,8/4/2025 12:08,ModernHR,Disk cleanup completed successfully.,System Notification,regex
2183,3/13/2025 7:23,ThirdPartyAPI,File data_7589.csv uploaded successfully by us...,System Notification,regex
2172,6/8/2025 20:38,BillingSystem,Backup ended at 2025-07-05 07:44:33.,System Notification,regex
1406,5/20/2025 11:05,ThirdPartyAPI,Backup started at 2025-09-05 04:17:13.,System Notification,regex
151,7/12/2025 13:01,ThirdPartyAPI,Backup completed successfully.,System Notification,regex
1168,4/6/2025 9:59,BillingSystem,Backup completed successfully.,System Notification,regex
1983,11/15/2025 16:09,BillingSystem,Backup ended at 2025-06-08 23:24:35.,System Notification,regex


In [9]:
df['log_message'] = df['log_message'].fillna('').astype(str)


In [10]:
df[df.log_message.str.startswith("System reboot initiated by user")]

,timestamp,source,log_message,target_label,complexity
36,11/19/2025 13:14,BillingSystem,System reboot initiated by user User243.,System Notification,regex
92,12/4/2025 21:20,BillingSystem,System reboot initiated by user User471.,System Notification,regex
139,5/8/2025 16:34,ModernHR,System reboot initiated by user User216.,System Notification,regex
140,9/11/2025 8:49,AnalyticsEngine,System reboot initiated by user User639.,System Notification,regex
161,3/31/2025 19:40,BillingSystem,System reboot initiated by user User819.,System Notification,regex
163,6/6/2025 15:29,BillingSystem,System reboot initiated by user User938.,System Notification,regex
307,4/12/2025 0:41,BillingSystem,System reboot initiated by user User929.,System Notification,regex
365,10/20/2025 22:32,ModernHR,System reboot initiated by user User533.,System Notification,regex
508,4/15/2025 2:04,ThirdPartyAPI,System reboot initiated by user User591.,System Notification,regex
552,9/22/2025 20:54,ModernHR,System reboot initiated by user User421.,System Notification,regex


CLUSTERING

In [11]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for log_message
embeddings = model.encode(df['log_message'].tolist())



C:\Users\fyraa\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
embeddings[:2]

array([[-1.02939680e-01,  3.35459560e-02, -2.20260359e-02,
         1.55098806e-03, -9.86920204e-03, -1.78956300e-01,
        -6.34410605e-02, -6.01761676e-02,  2.81108692e-02,
         5.99619746e-02, -1.72618758e-02,  1.43365108e-03,
        -1.49560079e-01,  3.15280934e-03, -5.66031151e-02,
         2.71685608e-02, -1.49890641e-02, -3.54037397e-02,
        -3.62936519e-02, -1.45410905e-02, -5.61487628e-03,
         8.75538737e-02,  4.55120914e-02,  2.50963476e-02,
         1.00187594e-02,  1.24266604e-02, -1.39923587e-01,
         7.68696070e-02,  3.14095728e-02, -4.15252987e-03,
         4.36902978e-02,  1.71250496e-02, -8.00951049e-02,
         5.74006774e-02,  1.89091545e-02,  8.55262130e-02,
         3.96398902e-02, -1.34371817e-01, -1.44364324e-03,
         3.06707597e-03,  1.76854074e-01,  4.44891443e-03,
        -1.69274714e-02,  2.24267151e-02, -4.35050540e-02,
         6.09034253e-03, -9.98167880e-03, -6.23972230e-02,
         1.07371844e-02, -6.04893221e-03, -7.14660734e-0

In [13]:
# Apply DBSCAN
# Adjusting eps from 0.5 to 0.2 and min samples from 5 to 1 gave us better results of clustering
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)
df['cluster'] =  clusters

df['is_anomaly'] = df['target_label'].isin(
    ['Critical Error', 'Error', 'Security Alert', 'Workflow Error', 'Deprecation Warning', 'Warning'])


df.head(20)

,timestamp,source,log_message,target_label,complexity,cluster,is_anomaly
0,2025/06/27 07:20:25,ModernCRM,"nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" status: 200 len: 1893 time: 0.2675118",HTTP Status,bert,0,False
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,True
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,True
3,2025-07-12 00:24:16,ModernHR,"nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" HTTP status code - 200 len: 211 time: 0.0968180",HTTP Status,bert,0,False
4,2025-06-02 18:25:23,BillingSystem,"nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" RCODE 200 len: 1874 time: 0.2280791",HTTP Status,bert,0,False
5,2025-10-09 10:30:31,ModernHR,"nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" Return code: 200 len: 1874 time: 0.2131531",HTTP Status,bert,0,False
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert,3,True
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by user User953.,System Notification,regex,4,False
8,2025-02-12 10:42:29,ThirdPartyAPI,"nova.compute.claims [req-a07ac654-8e81-416d-bfbb-189116b07969 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] [instance: bf8c824d-f099-4433-a41e-e3da7578262e] Total memory: 64172 MB, used: 512.00 MB",Resource Usage,bert,5,False
9,2025-03-30 04:01:45,ModernHR,"nova.osapi_compute.wsgi.server [req-2bf7cfee-a236-42f3-8fb1-96fefab0b302 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" RCODE 200 len: 1874 time: 0.1794369",HTTP Status,bert,0,False


In [18]:
print("Cluster 0 Content:")
print(df[df['cluster'] == 2])


Cluster 0 Content:
Empty DataFrame
Columns: [timestamp, source, log_message, target_label, complexity, cluster, is_anomaly]
Index: []


In [108]:
print(df[df['is_anomaly'] == True])


                timestamp           source  \
1         1/14/2025 23:07        ModernCRM   
2          1/17/2025 1:29  AnalyticsEngine   
6          3/1/2025 19:14         ModernHR   
10         8/9/2025 18:58        ModernCRM   
11        6/15/2025 11:44         ModernHR   
...                   ...              ...   
2420      12/3/2025 17:17            Linux   
2421      1/10/2025 23:50         ModernHR   
2423  2025/11/01 22:14:03   PaymentGateway   
2424   1/23/2025 10:15:33   SecuritySystem   
2425   1/13/2024 10:16:43   SecuritySystem   

                                                                  log_message  \
1                              Email service experiencing issues with sending   
2                                   Unauthorized access to data was attempted   
6                                   Shard 6 replication task ended in failure   
10                                   Email server encountered a sending fault   
11                            Critical sys

In [109]:
print(df['is_anomaly'].sum())


725


In [110]:
print(df[df['is_anomaly'] == True]['target_label'].unique())


['Critical Error' 'Security Alert' 'Error' 'Workflow Error'
 'Deprecation Warning' 'Warning']


In [131]:
anomalies_df = df[df['is_anomaly'] == True]
anomalies_df.to_csv('anomalies.csv', index=False)
anomalies_df.to_json('anomalies.json', orient='records', lines=True)


In [112]:
print(df.head())  # View the cleaned rows
print("**************************************************")
print(df.shape)  # Check the dimensions
print("**************************************************")
print(df.isnull().sum())  # Check for remaining null/missing values


             timestamp           source  \
0  2025/06/27 07:20:25        ModernCRM   
1      1/14/2025 23:07        ModernCRM   
2       1/17/2025 1:29  AnalyticsEngine   
3  2025-07-12 00:24:16         ModernHR   
4  2025-06-02 18:25:23    BillingSystem   

                                                                                                                                                                                                                                                                        log_message  \
0             nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118   
1                                                                                                                                                                                      

In [113]:
cluster_counts = df['cluster'].value_counts()
for cluster, count in cluster_counts.items():
    if count > 10:
        print(f"Cluster {cluster} (Count: {count}):")
        print(df[df['cluster'] == cluster]['log_message'].head(5).tolist())
        print("\n")

Cluster 0 (Count: 1017):
['nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118', 'nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180', 'nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791', 'nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54

In [114]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"HTTP/\d\.\d\ \d{3} .*": "HTTP Status",
        r"IP .* blocked due to potential attack" : "HTTP Status",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action",
        #Linux Logs
         r"sshd\[\d+\]: Accepted (password|publickey) for .* from .* port \d+ ssh2": "System Notification",
        r"sshd\[\d+\]: Failed password for .* from .* port \d+ ssh2": "Error",
        r"kernel: \[.*\] .*error.*": "Error",
        r"kernel: \[.*\] .*warn.*": "Warning",
        r"kernel: \[.*\] .*": "System Notification",
        r"CRON\[\d+\]: .* \(.*\)": "Linux Logs",
        r"systemd\[\d+\]: .*start.*": "System Notification",# Matches "Started" "Starting"
        r"systemd\[\d+\]: .*stop.*": "System Notification", # Matches "stopped" "Stopping"
        r"ufs: \[warn\].*": "Warning",
        r"kernel: \[.*\] CPU\d+: .*": "Warning",
        r"kernel: \[.*\] Memory allocation error.*": "Error",
        r"su: (FAILED session|FAILED|session opened) for user .* by .*": "Error"

    }

    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE): # searching pattern from log_message and ignoring uppercase or lowercase
            return label
    return None #dosen't return any value

In [115]:
classify_with_regex("ufs: [warn] Disk space running low on /dev/sda1.")

'Warning'

In [116]:
classify_with_regex("User User94 logged in.")

'User Action'

In [117]:
classify_with_regex("User User94 logged out.")


'User Action'

In [118]:
classify_with_regex("IP 192.168.133.114 blocked due to potential attack")

'HTTP Status'

In [119]:
classify_with_regex("User User94 logged OUt.")


'User Action'

In [120]:
classify_with_regex("User  OUt.")


In [121]:
df.shape

(2426, 7)

In [122]:
#Apply Regex classification
df['regex_label'] = df['log_message'].apply(classify_with_regex)
df[df.regex_label.isna()].shape

(1912, 8)

CLASSIFICATION USING EMBEDDINGS

In [123]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex

,timestamp,source,log_message,target_label,complexity,cluster,is_anomaly,regex_label
0,2025/06/27 07:20:25,ModernCRM,"nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" status: 200 len: 1893 time: 0.2675118",HTTP Status,bert,0,False,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,True,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,True,None
3,2025-07-12 00:24:16,ModernHR,"nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" HTTP status code - 200 len: 211 time: 0.0968180",HTTP Status,bert,0,False,None
4,2025-06-02 18:25:23,BillingSystem,"nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 ""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1"" RCODE 200 len: 1874 time: 0.2280791",HTTP Status,bert,0,False,None
...,...,...,...,...,...,...,...,...
2421,1/10/2025 23:50,ModernHR,Detected unauthorized access to confidential HR records (user: User568),Security Alert,bert,142,True,None
2422,2025/12/05 15:45:12,AnalyticsEngine,"File processing completed. Total records: 10,492, Errors: 0",System Notification,bert,143,False,None
2423,2025/11/01 22:14:03,PaymentGateway,Transaction ID TX7845 failed due to insufficient funds,Critical Error,bert,144,True,None
2424,1/23/2025 10:15:33,SecuritySystem,Firewall rule added: Restrict traffic to IP 192.168.1.1,Security Alert,bert,145,True,None


In [124]:
#every label that  contains under 5 values
small_target_labels = df_non_regex['target_label'].value_counts()
small_target_labels = small_target_labels[small_target_labels <= 5]
print(small_target_labels.index.tolist())

['Workflow Error', 'Deprecation Warning', 'System Notification']


In [125]:
df_non_legacy = df_non_regex[df_non_regex.source!='LegacyCRM']
df_non_legacy.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'PaymentGateway', 'SecuritySystem'], dtype=object)

In [126]:
missing_rows = df_non_legacy[df_non_legacy['source'].isna()]
print(missing_rows)


Empty DataFrame
Columns: [timestamp, source, log_message, target_label, complexity, cluster, is_anomaly, regex_label]
Index: []


In [127]:
# Generate embeddings for log_message
filtered_embeddings = model.encode(df_non_legacy['log_message'].tolist())
filtered_embeddings[:2]

array([[-1.02939717e-01,  3.35459895e-02, -2.20260434e-02,
         1.55100622e-03, -9.86912940e-03, -1.78956345e-01,
        -6.34409934e-02, -6.01761267e-02,  2.81108730e-02,
         5.99619709e-02, -1.72618106e-02,  1.43368833e-03,
        -1.49559975e-01,  3.15280259e-03, -5.66031225e-02,
         2.71685645e-02, -1.49890687e-02, -3.54037769e-02,
        -3.62936147e-02, -1.45410709e-02, -5.61491819e-03,
         8.75539333e-02,  4.55120951e-02,  2.50963438e-02,
         1.00188032e-02,  1.24267070e-02, -1.39923558e-01,
         7.68695921e-02,  3.14095505e-02, -4.15250845e-03,
         4.36903723e-02,  1.71250105e-02, -8.00951421e-02,
         5.74006326e-02,  1.89091861e-02,  8.55261832e-02,
         3.96398939e-02, -1.34371832e-01, -1.44363695e-03,
         3.06711602e-03,  1.76854104e-01,  4.44891676e-03,
        -1.69273838e-02,  2.24266760e-02, -4.35050540e-02,
         6.09031972e-03, -9.98173840e-03, -6.23971745e-02,
         1.07371574e-02, -6.04891405e-03, -7.14659989e-0

TRAINING THE LOGISTIC REGRESSION MODEL

In [128]:
X = filtered_embeddings
y = df_non_legacy['target_label']

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)




                precision    recall  f1-score   support

Critical Error       0.96      1.00      0.98        54
         Error       0.98      0.96      0.97        53
   HTTP Status       1.00      1.00      1.00       309
Resource Usage       1.00      1.00      1.00        51
Security Alert       1.00      0.99      1.00       105

      accuracy                           0.99       572
     macro avg       0.99      0.99      0.99       572
  weighted avg       0.99      0.99      0.99       572



In [129]:
import joblib

joblib.dump(clf, '../models/log_classifier.joblib')

['../models/log_classifier.joblib']

In [130]:
joblib.dump(dbscan, '../models/dbscan_model.joblib')

print("DBSCAN model saved successfully!")


DBSCAN model saved successfully!
